In [ ]:
# CELL 1: Install dependencies
import sys, subprocess
pkgs = ["geopandas", "shapely", "requests", "ipywidgets", "folium", "pyproj"]
subprocess.check_call([sys.executable, "-m", "pip", "install"] + pkgs + ["--quiet"])
print("Done!")

In [ ]:
# CELL 2: Imports and configuration
import os
import zipfile
import requests
import geopandas as gpd
import pandas as pd
import ipywidgets as widgets
from shapely.geometry import box, LineString, MultiLineString
from shapely.ops import unary_union
from pathlib import Path
from IPython.display import display, clear_output, HTML

# Blue Habitats - Harris et al. (2014) Global Seafloor Geomorphic Features Map
# Licensed under Creative Commons Attribution 4.0 International License
# Reference: Harris, P.T., Macmillan-Lawler, M., Rupp, J. and Baker, E.K. 2014.
#            Geomorphology of the oceans. Marine Geology, 352: 4-24.

DROPBOX_URL = "https://www.dropbox.com/scl/fi/mm01uhaa0ohyfitckmru7/global-seafloor-geomorphic-features-map.zip?rlkey=so36tzk2mfq6344xioq4v92y0&dl=1"

# Available geomorphic features in the dataset
FEATURES = [
    "shelf",           # Continental shelf
    "slope",           # Continental slope  
    "rise",            # Continental rise
    "abyss",           # Abyssal plains
    "basin",           # Ocean basins
    "canyon",          # Submarine canyons
    "escarpment",      # Escarpments
    "fan",             # Submarine fans
    "glacial_trough",  # Glacial troughs
    "guyot",           # Guyots (flat-topped seamounts)
    "hadal",           # Hadal zones (trenches >6000m)
    "plateau",         # Submarine plateaus
    "ridge",           # Oceanic ridges
    "rift_valley",     # Rift valleys
    "seamount",        # Seamounts
    "sill",            # Sills
    "spreading_ridge", # Mid-ocean spreading ridges
    "terrace",         # Terraces
    "trench",          # Ocean trenches
    "trough",          # Troughs
    "bridge"           # Land bridges
]

print("Harris et al. (2014) - Global Seafloor Geomorphic Features")
print(f"Available features: {len(FEATURES)}")
print("Data source: https://bluehabitats.org")

In [ ]:
# CELL 3: Interface

def download_and_extract(url, dest_folder, progress=None):
    """Download and extract the Harris et al. dataset."""
    dest = Path(dest_folder)
    zip_path = dest / "harris_geomorphology.zip"
    extract_path = dest / "harris_geomorphology"
    
    # Check if already extracted
    if extract_path.exists() and any(extract_path.glob("*.shp")):
        print(f"Data already exists in {extract_path}")
        return extract_path
    
    dest.mkdir(parents=True, exist_ok=True)
    extract_path.mkdir(parents=True, exist_ok=True)
    
    # Download
    print(f"Downloading dataset (~389 MB)...")
    print("This may take several minutes depending on your connection.")
    
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    
    if progress:
        progress.max = total_size if total_size > 0 else 100
        progress.value = 0
    
    downloaded = 0
    with open(zip_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)
                downloaded += len(chunk)
                if progress and total_size > 0:
                    progress.value = downloaded
    
    print(f"Download complete: {downloaded / 1e6:.1f} MB")
    
    # Extract
    print("Extracting files...")
    with zipfile.ZipFile(zip_path, 'r') as zf:
        zf.extractall(extract_path)
    
    # Clean up zip
    zip_path.unlink()
    print(f"Extracted to {extract_path}")
    
    return extract_path


def find_shapefile(data_path, feature_name):
    """Find shapefile for a given feature."""
    data_path = Path(data_path)
    
    # Try different naming patterns
    patterns = [
        f"*{feature_name}*.shp",
        f"*{feature_name.replace('_', '')}*.shp",
        f"*{feature_name.replace('_', ' ')}*.shp",
    ]
    
    for pattern in patterns:
        matches = list(data_path.rglob(pattern))
        if matches:
            return matches[0]
    
    return None


def list_available_shapefiles(data_path):
    """List all available shapefiles in the dataset."""
    data_path = Path(data_path)
    shapefiles = list(data_path.rglob("*.shp"))
    return {shp.stem: shp for shp in shapefiles}


def clip_to_bbox(gdf, bbox):
    """Clip GeoDataFrame to bounding box."""
    minlon, minlat, maxlon, maxlat = bbox
    clip_box = box(minlon, minlat, maxlon, maxlat)
    return gdf.clip(clip_box)


def extract_shelf_break_line(shelf_gdf, slope_gdf, bbox=None):
    """Extract shelf break line from shelf/slope boundary."""
    print("Extracting shelf break line from shelf/slope boundary...")
    
    # Clip first if bbox provided
    if bbox:
        shelf_gdf = clip_to_bbox(shelf_gdf, bbox)
        slope_gdf = clip_to_bbox(slope_gdf, bbox)
    
    if shelf_gdf.empty or slope_gdf.empty:
        print("No shelf or slope features in this area.")
        return None
    
    # Union all geometries
    shelf_union = unary_union(shelf_gdf.geometry)
    slope_union = unary_union(slope_gdf.geometry)
    
    # Get intersection (shared boundary)
    shelf_break = shelf_union.intersection(slope_union)
    
    if shelf_break.is_empty:
        # Try boundary intersection
        shelf_break = shelf_union.boundary.intersection(slope_union.boundary)
    
    if shelf_break.is_empty:
        print("Could not extract shelf break line.")
        return None
    
    # Convert to GeoDataFrame
    gdf = gpd.GeoDataFrame(
        {'feature': ['shelf_break'], 'source': ['Harris et al. 2014']},
        geometry=[shelf_break],
        crs=shelf_gdf.crs
    )
    
    print(f"Shelf break line extracted successfully.")
    return gdf


def process_features(data_path, features, bbox, outdir, extract_break=False, prog=None):
    """Process selected features."""
    data_path = Path(data_path)
    outpath = Path(outdir)
    outpath.mkdir(parents=True, exist_ok=True)
    
    available = list_available_shapefiles(data_path)
    print(f"Found {len(available)} shapefiles in dataset")
    
    if prog:
        prog.max = len(features) + (1 if extract_break else 0)
        prog.value = 0
    
    results = []
    shelf_gdf = None
    slope_gdf = None
    
    for i, feature in enumerate(features):
        shp_path = find_shapefile(data_path, feature)
        
        if not shp_path:
            # Try to find by listing available
            for name, path in available.items():
                if feature.lower() in name.lower():
                    shp_path = path
                    break
        
        if not shp_path:
            print(f"  SKIP {feature}: shapefile not found")
            continue
        
        try:
            print(f"  Processing {feature}...")
            gdf = gpd.read_file(shp_path)
            
            # Store for shelf break extraction
            if 'shelf' in feature.lower() and 'shelf' in shp_path.stem.lower():
                if 'slope' not in shp_path.stem.lower():
                    shelf_gdf = gdf.copy()
            if 'slope' in feature.lower() or 'slope' in shp_path.stem.lower():
                slope_gdf = gdf.copy()
            
            # Clip to bbox
            if bbox:
                gdf_clipped = clip_to_bbox(gdf, bbox)
            else:
                gdf_clipped = gdf
            
            if gdf_clipped.empty:
                print(f"    No features in selected area")
                continue
            
            # Save
            out_file = outpath / f"harris_{feature}.shp"
            gdf_clipped.to_file(out_file)
            print(f"    OK: {len(gdf_clipped)} features -> {out_file.name}")
            results.append((feature, len(gdf_clipped), out_file))
            
        except Exception as e:
            print(f"    FAIL: {str(e)[:60]}")
        
        if prog:
            prog.value = i + 1
    
    # Extract shelf break line if requested
    if extract_break:
        if shelf_gdf is None:
            # Load shelf if not already loaded
            shp_path = find_shapefile(data_path, "shelf")
            if shp_path:
                shelf_gdf = gpd.read_file(shp_path)
        
        if slope_gdf is None:
            # Load slope if not already loaded
            shp_path = find_shapefile(data_path, "slope")
            if shp_path:
                slope_gdf = gpd.read_file(shp_path)
        
        if shelf_gdf is not None and slope_gdf is not None:
            break_gdf = extract_shelf_break_line(shelf_gdf, slope_gdf, bbox)
            if break_gdf is not None:
                out_file = outpath / "harris_shelf_break_line.shp"
                break_gdf.to_file(out_file)
                print(f"    OK: Shelf break line -> {out_file.name}")
                results.append(('shelf_break_line', 1, out_file))
        else:
            print("  Could not extract shelf break: shelf or slope data missing")
        
        if prog:
            prog.value = prog.max
    
    return results


# Folder browser
class FolderBrowser:
    def __init__(self, start='.'):
        self.cur = Path(start).resolve()
        self.sel = self.cur
        self.html = widgets.HTML(f"<code>{self.cur}</code>")
        self.dd = widgets.Select(options=self._list(), layout=widgets.Layout(width='100%', height='100px'))
        self.b_up = widgets.Button(description='Up', layout=widgets.Layout(width='60px'))
        self.b_in = widgets.Button(description='Enter', layout=widgets.Layout(width='70px'))
        self.b_sel = widgets.Button(description='Select', button_style='success', layout=widgets.Layout(width='70px'))
        self.txt = widgets.Text(placeholder='new folder', layout=widgets.Layout(width='120px'))
        self.b_new = widgets.Button(description='+', layout=widgets.Layout(width='40px'))
        self.selhtml = widgets.HTML(f"Selected: <code>{self.sel}</code>")
        self.b_up.on_click(lambda b: self._up())
        self.b_in.on_click(lambda b: self._enter())
        self.b_sel.on_click(lambda b: self._select())
        self.b_new.on_click(lambda b: self._create())
        self.w = widgets.VBox([widgets.HTML("<b>Output Folder</b>"), self.html, self.dd,
                               widgets.HBox([self.b_up, self.b_in, self.b_sel, self.txt, self.b_new]), self.selhtml])
    def _list(self):
        try:
            return ['.'] + [x.name for x in sorted(self.cur.iterdir()) if x.is_dir() and not x.name.startswith('.')]
        except:
            return ['.']
    def _refresh(self):
        self.html.value = f"<code>{self.cur}</code>"
        self.dd.options = self._list()
    def _up(self):
        if self.cur.parent != self.cur:
            self.cur = self.cur.parent
            self._refresh()
    def _enter(self):
        if self.dd.value and self.dd.value != '.':
            p = self.cur / self.dd.value
            if p.is_dir():
                self.cur = p
                self._refresh()
    def _select(self):
        self.sel = self.cur
        self.selhtml.value = f"Selected: <code>{self.sel}</code>"
    def _create(self):
        n = self.txt.value.strip()
        if n:
            p = self.cur / n
            p.mkdir(parents=True, exist_ok=True)
            self.cur = p
            self.sel = p
            self.txt.value = ''
            self._refresh()
            self.selhtml.value = f"Created: <code>{self.sel}</code>"
    def path(self):
        return str(self.sel)


# UI Components
fb = FolderBrowser('.')

# Region selection
w_lat = widgets.FloatRangeSlider(
    value=[-60, 10], min=-90, max=90, step=0.5,
    description='Lat:', layout=widgets.Layout(width='50%'),
    continuous_update=False
)
w_lon = widgets.FloatRangeSlider(
    value=[-70, 30], min=-180, max=180, step=0.5,
    description='Lon:', layout=widgets.Layout(width='50%'),
    continuous_update=False
)

# Feature selection
w_features = widgets.SelectMultiple(
    options=FEATURES,
    value=['shelf', 'slope'],
    description='Features:',
    layout=widgets.Layout(width='50%', height='200px')
)

# Shelf break extraction option
w_extract_break = widgets.Checkbox(
    value=True,
    description='Extract Shelf Break Line (boundary between shelf and slope)',
    layout=widgets.Layout(width='100%')
)

# Progress and buttons
w_prog = widgets.IntProgress(min=0, max=1, description='Progress:', layout=widgets.Layout(width='50%'))
w_btn_download = widgets.Button(
    description='1. DOWNLOAD DATASET',
    button_style='info',
    layout=widgets.Layout(width='49%', height='40px')
)
w_btn_process = widgets.Button(
    description='2. PROCESS & CLIP',
    button_style='success',
    layout=widgets.Layout(width='49%', height='40px')
)
w_log = widgets.Output(layout=widgets.Layout(border='1px solid #ccc', max_height='400px', overflow='auto'))

# State
data_path = [None]  # Use list to allow modification in nested function

def on_download(b):
    with w_log:
        clear_output()
        print("Starting download from Blue Habitats (Dropbox)...")
        print("Reference: Harris et al. (2014) Marine Geology 352: 4-24")
        print("License: CC BY 4.0\n")
        try:
            data_path[0] = download_and_extract(DROPBOX_URL, fb.path(), w_prog)
            print("\nDataset ready! You can now process features.")
            
            # List available shapefiles
            available = list_available_shapefiles(data_path[0])
            print(f"\nAvailable shapefiles ({len(available)}):")
            for name in sorted(available.keys()):
                print(f"  - {name}")
        except Exception as e:
            print(f"\nError: {e}")

def on_process(b):
    with w_log:
        clear_output()
        
        # Check if data exists
        if data_path[0] is None:
            # Try to find existing data
            check_path = Path(fb.path()) / "harris_geomorphology"
            if check_path.exists() and any(check_path.glob("*.shp")):
                data_path[0] = check_path
            else:
                print("Please download the dataset first!")
                return
        
        features = list(w_features.value)
        if not features and not w_extract_break.value:
            print("Select at least one feature or enable shelf break extraction!")
            return
        
        bbox = (w_lon.value[0], w_lat.value[0], w_lon.value[1], w_lat.value[1])
        print(f"Processing features: {', '.join(features) if features else 'none'}")
        print(f"Bounding box: Lat[{bbox[1]}:{bbox[3]}] Lon[{bbox[0]}:{bbox[2]}]")
        print(f"Extract shelf break line: {w_extract_break.value}")
        print(f"Output: {fb.path()}\n")
        
        results = process_features(
            data_path[0],
            features,
            bbox,
            fb.path(),
            w_extract_break.value,
            w_prog
        )
        
        print(f"\n" + "="*50)
        print(f"SUMMARY: {len(results)} files created")
        for feat, count, path in results:
            print(f"  {feat}: {count} features -> {path.name}")

w_btn_download.on_click(on_download)
w_btn_process.on_click(on_process)

# Display interface
display(widgets.VBox([
    widgets.HTML("<h2>Harris et al. (2014) - Global Seafloor Geomorphic Features</h2>"),
    widgets.HTML("<i>Reference: Harris, P.T., Macmillan-Lawler, M., Rupp, J. and Baker, E.K. 2014. "
                 "Geomorphology of the oceans. Marine Geology, 352: 4-24.</i>"),
    widgets.HTML("<i>License: <a href='https://creativecommons.org/licenses/by/4.0/'>CC BY 4.0</a> | "
                 "Source: <a href='https://bluehabitats.org'>bluehabitats.org</a></i><br>"),
    fb.w,
    widgets.HTML("<br><b>Region of Interest</b>"),
    w_lat, w_lon,
    widgets.HTML("<br><b>Geomorphic Features</b> (Ctrl+click to select multiple)"),
    w_features,
    w_extract_break,
    widgets.HTML("<br>"),
    widgets.HBox([w_btn_download, w_btn_process]),
    w_prog,
    widgets.HTML("<b>Log:</b>"),
    w_log
]))

In [ ]:
# CELL 4: Quick visualization (optional)
# Run this cell after processing to visualize results

import matplotlib.pyplot as plt

def plot_results(output_folder):
    """Plot all shapefiles in the output folder."""
    output_path = Path(output_folder)
    shapefiles = list(output_path.glob("harris_*.shp"))
    
    if not shapefiles:
        print("No Harris shapefiles found in output folder.")
        return
    
    fig, ax = plt.subplots(figsize=(12, 8))
    
    colors = plt.cm.tab20.colors
    
    for i, shp in enumerate(shapefiles):
        gdf = gpd.read_file(shp)
        label = shp.stem.replace('harris_', '')
        
        if 'line' in label.lower() or gdf.geometry.iloc[0].geom_type in ['LineString', 'MultiLineString']:
            gdf.plot(ax=ax, color=colors[i % len(colors)], linewidth=2, label=label)
        else:
            gdf.plot(ax=ax, color=colors[i % len(colors)], alpha=0.5, edgecolor='black', label=label)
    
    ax.legend(loc='best')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title('Harris et al. (2014) - Geomorphic Features')
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# Uncomment and modify the path to visualize:
# plot_results("/path/to/your/output")

In [ ]:
# CELL 5: Alternative - Extract shelf break using polygon boundaries directly
# This cell provides a more robust method using polygon boundary extraction

def extract_shelf_break_boundary(data_path, bbox=None, output_path=None):
    """
    Extract shelf break line by finding the outer boundary of shelf polygons.
    The shelf break is defined as the edge of the continental shelf.
    """
    from shapely.ops import linemerge
    
    data_path = Path(data_path)
    
    # Find shelf shapefile
    shelf_shp = None
    for shp in data_path.rglob("*.shp"):
        if 'shelf' in shp.stem.lower() and 'valley' not in shp.stem.lower():
            shelf_shp = shp
            break
    
    if shelf_shp is None:
        print("Shelf shapefile not found!")
        return None
    
    print(f"Loading shelf data from {shelf_shp.name}...")
    shelf_gdf = gpd.read_file(shelf_shp)
    
    # Clip to bbox if provided
    if bbox:
        minlon, minlat, maxlon, maxlat = bbox
        clip_box = box(minlon, minlat, maxlon, maxlat)
        shelf_gdf = shelf_gdf.clip(clip_box)
    
    if shelf_gdf.empty:
        print("No shelf features in selected area.")
        return None
    
    print(f"Processing {len(shelf_gdf)} shelf polygons...")
    
    # Get all boundaries
    boundaries = shelf_gdf.geometry.boundary
    
    # Merge into single geometry
    all_boundaries = unary_union(boundaries.tolist())
    
    # Try to merge lines
    try:
        merged = linemerge(all_boundaries)
    except:
        merged = all_boundaries
    
    # Create GeoDataFrame
    result = gpd.GeoDataFrame(
        {
            'feature': ['shelf_boundary'],
            'source': ['Harris et al. 2014'],
            'description': ['Continental shelf boundary (includes shelf break)']
        },
        geometry=[merged],
        crs=shelf_gdf.crs
    )
    
    if output_path:
        output_path = Path(output_path)
        result.to_file(output_path / "harris_shelf_boundary.shp")
        print(f"Saved to {output_path / 'harris_shelf_boundary.shp'}")
    
    return result

# Example usage:
# extract_shelf_break_boundary(
#     data_path="/path/to/harris_geomorphology",
#     bbox=(-70, -60, 30, 10),  # (minlon, minlat, maxlon, maxlat)
#     output_path="/path/to/output"
# )